# Capstone Project - Vegan Restaurant in Manhattan
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In recent years the interest in a vegan lifestyle has increased. There are a lot of reason
as: people concerned with animal rights, animal mistreatment, climate change,
health and others. Below are some statistics:

* If the world went vegan, it could save 8 million human lives by 2050, reduce greenhouse gas emissions by two thirds and lead to healthcare-related savings and avoided climate damages of $1.5 trillion. (https://www.oxfordmartin.ox.ac.uk/news/201603-plant-based-diets/ )

* Dairy cows have been bred to produce up to 10 times more milk than they naturally would. (https://www.ciwf.org.uk/media/5235185/the-life-of-dairy-cows.pdf )

* Male calves are of no use to the dairy industry and are less suitable for beef production. This means that every year around 95,000 male dairy calves are shot soon after birth and discarded as a by-product.
(https://www.theguardian.com/environment/2018/mar/26/dairy-dirty-secret-its-still-cheaperto-kill-male-calves-than-to-rear-them)

* Animal agriculture contributes an estimated 18% to total greenhouse gas emissions from the five major sectors for greenhouse gas reporting. For the agriculture sector alone, farmed animals constitute nearly 80% of all emissions.
(http://www.fao.org/3/a-a0701e.pdf ) [p. 112]

For more statistics about vegan, consult https://www.vegansociety.com/news/media/statistics

As the number of vegan people is growing up, it is interest using data to decide
where in the city of New York open an vegan restaurant. It is ideal to find some
heighborhood with few restaurants of the kind and close to a lot of people.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Vegan restaurants in the neighborhood, if any
* distance of neighborhood from city center

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**


### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods.

Let's first find the latitude & longitude of Manhattan.

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'X4D2O1VY0QPNU4RELECJCSB2UVAUYQNYOJCBTQBGDC0LYJIT' # your Foursquare ID
CLIENT_SECRET = 'DJYRZ1EQERBSC0NSAJSELZJKJPIMAKPMVAR0WK4VDADEKMIT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X4D2O1VY0QPNU4RELECJCSB2UVAUYQNYOJCBTQBGDC0LYJIT
CLIENT_SECRET:DJYRZ1EQERBSC0NSAJSELZJKJPIMAKPMVAR0WK4VDADEKMIT


Lets download the data from https://cocl.us/new_york_dataset.

In [ ]:
path = 'https://cocl.us/new_york_dataset'
df = pd.read_json(path)
print('Data downloaded!')

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [5]:
neighborhoods_data = newyork_data['features']

#### Tranform the data into a *pandas* dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.

In [10]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="manhattan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Create a map of Manhattan with neighborhoods superimposed on top.

In [12]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Now, let's get the top 100 venues that are in New York within a radius of 500 meters.

In [13]:
radius = 5000
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=X4D2O1VY0QPNU4RELECJCSB2UVAUYQNYOJCBTQBGDC0LYJIT&client_secret=DJYRZ1EQERBSC0NSAJSELZJKJPIMAKPMVAR0WK4VDADEKMIT&ll=40.7896239,-73.9598939&v=20180605&radius=5000&limit=100'

Send the GET request and examine the resutls

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e44453e963d29001be9d726'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'New York',
  'headerFullLocation': 'New York',
  'headerLocationGranularity': 'city',
  'totalResults': 236,
  'suggestedBounds': {'ne': {'lat': 40.83462394500005,
    'lng': -73.90056853843416},
   'sw': {'lat': 40.74462385499996, 'lng': -74.01921926156584}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba233dbf964a5206fe337e3',
       'name': 'East Meadow',
       'location': {'address': 'Central Park',
        'crossStreet': '5th Ave btwn 98th & 101st St',
        'lat': 40.79015961797437,
        'lng': -73.95549774169922,


We know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function.

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Hevans\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,East Meadow,Field,40.790160,-73.955498
1,The Jewish Museum,Museum,40.785276,-73.957411
2,Jacqueline Kennedy Onassis Reservoir,Reservoir,40.784519,-73.960966
3,K&D Wines & Spirits,Wine Shop,40.787096,-73.954261
4,Cooper Hewitt Smithsonian Design Museum,Art Museum,40.784333,-73.957765


## 2. Explore Neighborhoods in New York

#### Let's create a function to repeat the same process to all the neighborhoods in New York

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [18]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check the size of the resulting dataframe

In [19]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3323, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


Let's check how many venues were returned for each neighborhood

In [20]:
manhattan_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,4,4,4,4,4,4
Adult Boutique,2,2,2,2,2,2
Afghan Restaurant,1,1,1,1,1,1
African Restaurant,2,2,2,2,2,2
American Restaurant,81,81,81,81,81,81
Antique Shop,1,1,1,1,1,1
Arcade,1,1,1,1,1,1
Arepa Restaurant,2,2,2,2,2,2
Argentinian Restaurant,5,5,5,5,5,5


In [21]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 334 uniques categories.


Let's drop the venues that are not vegan/vegetarian restaurants.

In [22]:
vegan_data = manhattan_venues[manhattan_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant'].reset_index(drop=True)
vegan_data.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Orchard Grocer,40.717847,-73.990358,Vegetarian / Vegan Restaurant
1,Upper East Side,40.775639,-73.960508,Candle 79,40.774896,-73.958694,Vegetarian / Vegan Restaurant
2,Upper East Side,40.775639,-73.960508,Candle Cafe,40.771407,-73.959138,Vegetarian / Vegan Restaurant
3,Lenox Hill,40.768113,-73.958860,Candle Cafe,40.771407,-73.959138,Vegetarian / Vegan Restaurant
4,Upper West Side,40.787658,-73.977059,Peacefood Cafe,40.785166,-73.977064,Vegetarian / Vegan Restaurant


In [23]:
vegan_data = vegan_data.drop(['Venue Category', 'Neighborhood Latitude', 'Neighborhood Longitude'], axis=1)
vegan_data.head()

,Neighborhood,Venue,Venue Latitude,Venue Longitude
0,Chinatown,Orchard Grocer,40.717847,-73.990358
1,Upper East Side,Candle 79,40.774896,-73.958694
2,Upper East Side,Candle Cafe,40.771407,-73.959138
3,Lenox Hill,Candle Cafe,40.771407,-73.959138
4,Upper West Side,Peacefood Cafe,40.785166,-73.977064


In [24]:
vegan_data.shape

(24, 4)

There are only 24 vegan/vegetarian restaurants in Manhattan.

Let's plot the map with this restaurants.

In [25]:
# create map of Manhattan using latitude and longitude values
map_manhattan_vegan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(vegan_data['Venue Latitude'], vegan_data['Venue Longitude'], vegan_data['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_vegan)  
    
map_manhattan_vegan

Looking good. So now we have all the vegan/vegetarian restaurants in Manhattan area!

Looks like a few pockets of low restaurant density closest to city center can be found **south and north from Central Park**.

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Manhattan that have low restaurant density, particularly those with low number of Vegan/Vegetarian restaurants. We will limit our analysis to area 5km around Manhattan.

In first step we have collected the required **data: location and category of every restaurant within 5km from Manhattan**. We have also **identified Vegan/vegetarian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Manhattan - we will use **heatmaps** to identify a few promising areas close to center with low number of vegan/vegetarian restaurants and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Vegan/vegetarian restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of vegan/vegetarian restaurants in every area candidate**:

In [26]:
vegan_data.groupby('Neighborhood').count()

,Venue,Venue Latitude,Venue Longitude
Neighborhood,,,
Carnegie Hill,1,1,1
Chelsea,1,1,1
Chinatown,1,1,1
Clinton,1,1,1
East Village,3,3,3
Flatiron,2,2,2
Greenwich Village,1,1,1
Lenox Hill,1,1,1
Little Italy,1,1,1


What we have now is a clear indication of zones with low number of vegan/vegetarian restaurants in vicinity.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [ ]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = vegan_data[['Venue Latitude','Venue Longitude']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

In [27]:
good_latitudes = vegan_data['Venue Latitude'].values
good_longitudes = vegan_data['Venue Longitude'].values

In [ ]:
for label in manhattan_data['Neighborhood']:
    label = folium.Popup(label, parse_html=True)
    map_manhattan2 = folium.Map(location=[latitude, longitude], zoom_start=13)
    folium.TileLayer('cartodbpositron').add_to(map_manhattan2)
    #HeatMap(restaurant_latlons).add_to(map_manhattan2)
    folium.Circle([latitude, longitude], radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_manhattan2)
    folium.Marker([latitude, longitude]).add_to(map_manhattan2)
    for lon, lat in cluster_centers:
        folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_manhattan2) 
    for lat, lon in zip(good_latitudes, good_longitudes):
        folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_manhattan2)
    folium.GeoJson(newyork_data, style_function=boroughs_style, name='geojson').add_to(map_manhattan2)
map_manhattan2

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of vegan restaurants nearby, all zones being fairly close to city center. Although zones are shown on map with a radius of ~500 meters, their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. The neighborhoods of interest are: Morningside Heights, Manhattan Valley, East Harlem, Lincoln Square and Lenox Hill.

These zones have as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Manhattan, there are a low number of vegan/vegetarian restaurantes close to city center. Highest concentration of restaurants was detected south from Central Park, so we focused our attention to areas north and east, corresponding to boroughs Morningside Heights, Manhattan Valley, East Harlem, Lincoln Square and Lenox Hill.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! 

Purpose of this analysis was to only provide info on areas close to Manhattan but not crowded with existing vegan/vegetarian restaurants - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Manhattan areas close to center with low number of vegan/vegetarian restaurants in order to aid stakeholders in narrowing down the search for optimal location for a new Vegan restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.